In [ ]:
import numpy as np 
import pandas as pd 
import json
from tensorflow.keras.models import load_model
import re
import pandas as pd
import string
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM,Dropout,concatenate,UpSampling1D
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Activation, Embedding, LSTM,Dropout
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Flatten ,Embedding,Input,Conv1D,GlobalMaxPooling1D,Dropout,MaxPooling1D,Bidirectional
from keras.models import Sequential,Model

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
data=pd.read_csv('../input/commonlitreadabilityprize/train.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
excerpt=data['excerpt']

In [ ]:
tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(excerpt)
sequences = tokenizer.texts_to_sequences(excerpt)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            100,
                            input_length=500)

In [ ]:
data1 = pad_sequences(sequences, maxlen=500)

In [ ]:
def crt_model():
    inp1=Input(shape=(500,))
    l0=embedding_layer(inp1)
    
    t11=Conv1D(64,5,padding='valid', kernel_initializer='normal',activation='relu')(l0)
    t12=MaxPooling1D(2) (t11)
    t13=UpSampling1D(2)(t12)
    t14=concatenate([t11,t13])
    t15=Conv1D(128,5,padding='valid', kernel_initializer='normal',activation='relu')(t14)
    t15=Conv1D(256,5,padding='valid', kernel_initializer='normal',activation='relu')(t15)
    t15=Conv1D(512,5,padding='valid', kernel_initializer='normal',activation='relu')(t15)
    t15=Conv1D(1024,5,padding='valid', kernel_initializer='normal',activation='relu')(t15)
    t16=GlobalMaxPooling1D()(t15)
    
    
    t21=Conv1D(64,3,padding='valid', kernel_initializer='normal',activation='relu')(l0)
    t22=MaxPooling1D(2) (t21)
    t23=UpSampling1D(2)(t22)
    t24=concatenate([t21,t23])
    t25=Conv1D(128,3,padding='valid', kernel_initializer='normal',activation='relu')(t24)
    t25=Conv1D(256,3,padding='valid', kernel_initializer='normal',activation='relu')(t25)
    t25=Conv1D(512,3,padding='valid', kernel_initializer='normal',activation='relu')(t25)
    t25=Conv1D(1024,3,padding='valid', kernel_initializer='normal',activation='relu')(t25)
    t26=GlobalMaxPooling1D()(t25)
    
    
    t31=Conv1D(64,3,padding='valid', kernel_initializer='normal',activation='relu')(l0)
    t32=MaxPooling1D(2) (t31)
    t33=UpSampling1D(2)(t32)
    t34=concatenate([t31,t33])
    t35=Conv1D(128,3,padding='valid', kernel_initializer='normal',activation='relu')(t34)
    t35=Conv1D(256,3,padding='valid', kernel_initializer='normal',activation='relu')(t35)
    t35=Conv1D(512,3,padding='valid', kernel_initializer='normal',activation='relu')(t35)
    t35=Conv1D(1024,3,padding='valid', kernel_initializer='normal',activation='relu')(t35)
    t36=Bidirectional(LSTM(128))(t35)
    t36=Dense(128, kernel_initializer='normal',activation='relu')(t36)
    
    
    con = concatenate([t16,t26,t36])
    
   
    l5=Dense(256, kernel_initializer='normal',activation='relu')(con)
    l5=Dense(512, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(600, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(512, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(256, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(128, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(64, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(32, kernel_initializer='normal',activation='relu')(l5)
    l5=Dense(8, kernel_initializer='normal',activation='relu')(l5)
    l7=Dense(1, kernel_initializer='normal')(l5)
    model=Model(inputs=inp1, outputs=l7)
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[keras.metrics.MeanSquaredError()])
    return model


model1=crt_model()
model2=crt_model()
model3=crt_model()
model1.summary()

In [ ]:
Y=data['target']
Error=data['standard_error']


Y=np.asarray(Y)
Err=np.asarray(Error)


Y1=[]
Y2=[]
for i in range(len(Y)):
    Y1.append(Y[i]+Err[i])
    Y2.append(Y[i]-Err[i])
    
Y1=np.asarray(Y1)
Y2=np.asarray(Y2)

    

In [ ]:
hist=model1.fit(data1,Y,epochs=20, batch_size=256)

In [ ]:
hist2=model2.fit(data1,Y1,epochs=20, batch_size=256)

In [ ]:
hist3=model3.fit(data1,Y2,epochs=20, batch_size=256)

In [ ]:
dt=pd.read_csv('../input/commonlitreadabilityprize/test.csv')
excerpt_t=dt['excerpt']
tokenizer3 = Tokenizer(num_words=None)
tokenizer3.fit_on_texts(excerpt_t)
sequences_t = tokenizer3.texts_to_sequences(excerpt_t)

In [ ]:
data_t = pad_sequences(sequences_t, maxlen=500)

In [ ]:
data_t = np.asarray(data_t)

In [ ]:
ypred1=model1.predict(data_t)
ypred2=model2.predict(data_t)
ypred3=model3.predict(data_t)

In [ ]:
def pr(ypred1,ypred2,ypred3):
    y_pred=[]
    for i in range(len(ypred1)):
        y_pred.append((ypred1[i][0]+ypred2[i][0]+ypred3[i][0])/3)
    return y_pred
ypred=pr(ypred1,ypred2,ypred3)
ypred

In [ ]:
submission = pd.DataFrame({'id':dt.id,'target':ypred})
submission.to_csv('submission.csv',index=False)